In [1]:
import gym
from mountain_car_env import CTMountainCarEnv
from sindy_model import create_transitionfunction

In [3]:
model = create_transitionfunction(num_trajs=10)

Creating the transition function


/Users/agnimagarwal/Downloads/src2/notebooks/sindy_model.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(state_action), np.array(states), np.array(actions)


ValueError: setting an array element with a sequence.

In [ ]:
model.print()

In [ ]:
import numpy as np
pos, vel = -0.5, 0
positions, velocities = [pos], [vel]
gt_positions, gt_velocities = [pos], [vel]
t = 1000
for i in range(t):
    new_state = model.predict(np.array([[pos, vel]]), u=np.array([[1]]))
    pos, vel = new_state[0]
    positions.append(pos)
    velocities.append(vel)

In [ ]:
from sindy_model import test_transitionfunction
test_transitionfunction(model)

In [ ]:
sindy_env = Continuous_MountainCarEnv_SINDy(sindy_model=model)
gt_env = Continuous_MountainCarEnv_SINDy(sindy_model=None)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def runEpisodeBB(env, heuristic=True, agent=None):
    obs = env.reset()
    state_action = []
    states = []
    actions = []
    num_episodes = 0
    k = 2
    action = env.action_space.sample()
    action = env.action_space.sample()
    for i in range(1000):
        if heuristic:
            action = np.array([1])
        if agent == None:
            curr_action = action
            action = -1*action
            action = env.action_space.sample() if np.random.uniform() < 0.2 else action
        else:
            action = agent.select_action(obs, evaluate=True)
        #temp = 10 if action == 1 else -10
        state_action.append([*obs, action])
        states.append([*obs])
        actions.append(action)
        next_obs, reward, done = env.step(action)[:3]
        obs = np.copy(next_obs)
        if done:
            obs, _ = env.reset()
            num_episodes += 1
            if num_episodes == 1:
                break
    # print("Number of episodes in data collection ", num_episodes)
    return np.array(state_action), np.array(states), np.array(actions)

_, sindy_states, sindy_actions = runEpisodeBB(sindy_env)
_, gt_states, gt_actions = runEpisodeBB(gt_env)
plt.plot(gt_states[:,0])
plt.plot(sindy_states[:,0])

In [ ]:
plt.plot(gt_states[:,1])
plt.plot(sindy_states[:,1])

In [ ]:
gt_states

In [ ]:
import numpy as np
from stable_baselines import DDPG
from stable_baselines.ddpg.policies import MlpPolicy as MlpPolicyDDPG
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
board_dir = 'tb_ddpg'

tot_steps = int(1e6)
# the noise objects for DDPG
n_actions = env.action_space.shape[-1]
param_noise = None
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

model = DDPG(MlpPolicyDDPG, env, verbose=1, param_noise=param_noise, action_noise=action_noise, tensorboard_log=board_dir)
model.learn(total_timesteps=tot_steps)
model.save("ddpg_cartpole")